In [1]:
from pyspark.sql.functions import regexp_replace, trim, col, lower

def removePunctuation(column):
    return trim(regexp_replace(lower(trim(column)),'[^a-zA-Z0-9 ]',''))

#let's test it
sentenceDF = sqlContext.createDataFrame([('Hi, you!',),
                                         (' No under_score!',),
                                         (' *      Remove punctuation then spaces  * ',)], ['sentence'])
sentenceDF.show(truncate=False)
(sentenceDF
 .select(removePunctuation(col('sentence')).alias('sentence'))
 .show(truncate=False))


In [2]:
fileName = "dbfs:/databricks-datasets/cs100/lab1/data-001/shakespeare.txt"

shakespeareDF = sqlContext.read.text(fileName).select(removePunctuation(col('value')).alias('sentence'))
shakespeareDF.show(15, truncate=False)

In [3]:
def wordCount(wordListDF):
    return wordListDF.groupBy('word').count()

In [4]:
from pyspark.sql.functions import split, explode
shakeWordsDF = (shakespeareDF.select(explode(split(shakespeareDF.sentence,' ')).alias('word')))
shakeWordsDF = shakeWordsDF.filter(shakeWordsDF.word!='')
shakeWordsDF = shakeWordsDF.filter(shakeWordsDF.word in {'the','and'})

shakeWordsDFCount = shakeWordsDF.count()
print shakeWordsDFCount

In [5]:
# TODO: Replace <FILL IN> with appropriate code
from pyspark.sql.functions import desc
topWordsAndCountsDF = wordCount(shakeWordsDF).orderBy('count',ascending = False)
topWordsAndCountsDF.collect()

In [6]:
from pyspark.sql.types import BooleanType
stopwords = ['', ' ','most','much','never','nor','great','up','mine','who','can',u'the', u'and', u'i', u'to', u'of', u'a', u'you', u'the', u'and', u'i', u'to', u'of', u'a', u'you', u'my', u'in', u'that', u'is', u'not', u'with', u'me', u'it', u'for', u'be', u'his', u'your', u'this', u'but', u'he', u'have', u'as', u'thou', u'him', u'so', u'will', u'what', u'thy', u'all', u'her', u'no', u'do', u'by', u'shall', u'if', u'are', u'we', u'thee', u'our', u'on', u'good', u'now', u'sir', u'from', u'o', u'come', u'at', u'they', u'well', u'or', u'which', u'would', u'more', u'was', u'then', u'she', u'am', u'how', u'here', u'let' , u'their', u'love', u'when', u'them', u'ill', u'hath', u'than', u'man', u'an', u'there', u'one', u'go', u'upon', u'like', u'say', u'know', u'may', u'make', u'did', u'us', u'were', u'should', u'yet', u'must', u'why', u'see', u'had', u'tis', u'such', u'out', u'some', u'give', u'these', u'too', u'where']

isstopword_udf = udf(lambda w: w in stopwords, BooleanType())
slen = udf(lambda s: len(s), IntegerType())
topWordsAndCountsDF.filter( ~ isstopword_udf(topWordsAndCountsDF.word)).show()


In [7]:
import re
def removePunctuation(text):
    no_punc = re.sub(r'[^a-zA-Z0-9 ]+','',text)
    return re.sub(r'^\s+|\s+$','',no_punc).lower()
  
just_words = shakespeareDF.rdd.map(lambda r: removePunctuation(r['sentence']) ).flatMap(lambda s: s.split(' ')).filter(lambda r:r not in stopwords)
rdd_word_counts = just_words.map(lambda r: (r,1)).reduceByKey(lambda x,y:x+y)
rdd_word_counts.takeOrdered(15, key = lambda x: -x[1])